In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# from shutil import copyfile
# copyfile('/content/drive/MyDrive/FDL Project/Code/text_vectorization.py', 'text_vectorization.py')
# copyfile('/content/drive/MyDrive/FDL Project/Code/embedding.py', 'embedding.py')
# copyfile('/content/drive/MyDrive/FDL Project/Code/kfold_cv.py', 'kfold_cv.py')

In [ ]:
#!pip install iterative-stratification

In [56]:
import pandas as pd
import numpy as np
from tensorflow import keras
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import pickle
import importlib

#import nltk
#nltk.data.path.append('C:\\Users\\della\\anaconda3\\nltk-data')

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

#import pre_processing
#importlib.reload(pre_processing)

import text_vectorization
#importlib.reload(text_vectorization)

import embedding
importlib.reload(embedding)

import kfold_cv
importlib.reload(kfold_cv)

#import data_augmentation
#importlib.reload(data_augmentation)

<module 'kfold_cv' from 'c:\\Users\\Luca\\Desktop\\arXiv_classification\\kfold_cv.py'>

In [13]:
#train_set_processed = pre_processing.dataPreProcessing(train_set)
#train_set_processed.to_csv('data/train-set-cat1-processed.csv', index = False)

#train_set = pd.read_csv('/content/drive/MyDrive/FDL Project/Code/data/train-set-cat1-processed.csv')
train_set = pd.read_csv('data/train-set-cat1-processed.csv')

number_of_categories = len(train_set['label'].unique())
# One-hot encoding of the labels
label_train = to_categorical(train_set['label'], num_classes = number_of_categories, dtype = 'int64')

In [14]:
vocabulary_size = 50000
words_per_sentence = 200
embedding_dim = 100

In [38]:
text_vectorizer_keras = text_vectorization.createTextVectorizer(vocabulary_size, words_per_sentence, train_set['text'])
vocabulary_keras = text_vectorizer_keras.get_vocabulary()

embedding_matrix_glove = embedding.buildEmbeddingMatrix(embedding_dim, vocabulary_keras)
embedding_layer_glove = embedding.createEmbeddingLayer(embedding_matrix_glove, None)
embedding_layer_glove._name = 'GloVe'

feature_train_glove = text_vectorization.textVectorization(train_set['text'], text_vectorizer_keras)

In [52]:
# Input layer
input_layer = keras.Input(shape = (words_per_sentence,), dtype = 'int64')

# Embedding layer
x = embedding_layer_glove(input_layer)

# Hidden layers
x = keras.layers.Conv1D(filters = 128, kernel_size = 5, activation = 'relu')(x)
x = keras.layers.GlobalMaxPooling1D()(x)
x = keras.layers.Dropout(rate = 0.5)(x)

# Output layer
x = keras.layers.Dense(number_of_categories, activation = 'softmax')(x)
output_layer = x

# Neural network model
network = keras.Model(input_layer, output_layer, name = 'Conv1D')

In [57]:
hyperparams = [{
    'filters': 128,
    'kernel_size': 5,
    'rate': 0.5,
    'optimizer': 'rmsprop',
    'batch_size': 128
    }
    #  ,
    #   {
    #  'filters': 128,
    #  'kernel_size': 5,
    #  'rate': 0.5,
    #  'optimizer': 'adam',
    #  'batch_size': 128
    #  }
]
epochs = 1
k = 2

In [58]:
kfold_results = kfold_cv.kfoldCrossValidation(k, feature_train_glove, label_train, network, hyperparams, epochs)
print(kfold_results)

212/212 [==============================] - 40s 190ms/step - loss: 0.9060 - accuracy: 0.7050
[{'Network': 'Conv1D', 'Embedding': 'GloVe', 'filters': 128, 'kernel_size': 5, 'rate': 0.5, 'optimizer': 'rmsprop', 'batch_size': 128, 'loss_kfold': 0.797, 'accuracy_kfold': 0.736, 'best_number_epochs': 1}]
